# Pulling data and splitting into training and test sets 

In [1]:
import numpy as np
np.random.seed(42)
import matplotlib as mpl
from matplotlib import pyplot as plt

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('MNIST_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8)

In [3]:
X, y = mnist["data"], mnist["target"]
X.shape, y.shape

((70000, 784), (70000,))

In [4]:
split_index = 60000
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

# Created a KNN model, fit to training set, calculated accuracy

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [6]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [7]:
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
accuracy_score(y_test, y_pred)

0.96879999999999999

# Standardised Data and 